## Datos sobre la deuda internacional del Banco Mundial
<p>Los países, al igual que los seres humanos, se endeudan para gestionar su economía. Por ejemplo, el gasto en infraestructura es un ingrediente costoso que los ciudadanos de un país necesitan para llevar una vida cómoda. es la organización que otorga deuda a los países. 
<p>En este notebook, analizaremos los datos de deuda internacional recopilados por el  <a href="https://www.worldbank.org">Banco Mundial</a> . El conjunto de datos contiene información sobre el monto de deuda (en dólares estadounidenses) que tienen los países en desarrollo en varias categorías. Encontraremos las respuestas a preguntas como: </p>
<ul>
<li>¿Cuál es el monto total de deuda que tienen los países enumerados en el conjunto de datos?</li>
<li>¿Qué país posee la mayor cantidad de deuda y cuál es ese monto?</li>
<li>¿Cuál es el monto promedio de deuda que tienen los países en diferentes indicadores de deuda?</li>
</ul>
<p>La primera línea de código nos conecta a la base de datos <code>debts world bank</code> donde se encuentra la tabla <code>international_debt_clean</code>. Primero, seleccionemos <code>todas</em> las columnas de la tabla <code>international_debt_clean</code>. 

```sql
SELECT *
FROM international_debt_clean

## 1. Encontrando el número de países distintos
<p>En las primeras 13 filas, podemos ver la cantidad de deuda que tiene <em>Afganistán</em> en los diferentes indicadores de deuda. Pero no sabemos el número de países distintos que tenemos en la tabla. Hay repeticiones en los nombres de los países porque es más probable que un país tenga deuda en más de un indicador de deuda. </p>
<p>Sin un recuento de países únicos, no podremos realizar nuestros análisis estadísticos de manera holística. En esta sección, extraemos el número de países únicos presentes en la tabla. </p>

```sql
SELECT 
    COUNT(DISTINCT "country_name") AS total_distinct_countries

## 2. Conociendo los distintos indicadores de deuda
<p>Podemos ver que hay un total de 120 países presentes en la tabla. Como vimos en la primera sección, hay una columna llamada <code>indicator_name</code> que especifica brevemente el propósito de la deuda. Justo al lado de esa columna, hay otra columna llamada <code>indicator_code</code> que simboliza la categoría de estas deudas. Conocer estos distintos indicadores de deuda nos ayuda a comprender las áreas en las que un país puede estar endeudado. </p>

```sql
SELECT DISTINCT indicator_code AS distinct_debt_indicators
FROM international_debt_clean
ORDER BY distinct_debt_indicators

## 3. Totalizando las deuda de los países
<p>Como se mencionó anteriormente, la deuda financiera de un país en particular representa su estado económico. Pero si tuviéramos que proyectarla a escala global, ¿cómo lo abordaríamos?</p>
<p>Veamos el monto total de deuda (en USD) que tienen los diferentes países. Esto nos dará una idea de cómo se mantiene la economía general del mundo entero.</p>

```sql
SELECT 
    ROUND(SUM(debt) / 1000000, 2) AS total_debt  
FROM international_debt_clean;

## 4. País con la mayor deuda
<p>Ahora que tenemos el total exacto de las cantidades de deuda que deben varios países, averigüamos el país que posee la mayor cantidad de deuda junto con la cantidad. Para ello, debo <strong>tener en cuenta</strong> que esta deuda es la suma de diferentes deudas que debe un país en varias categorías. Esto me ayudará a comprender más sobre el país en términos de sus escenarios socioeconómicos. </p>

```sql
SELECT TOP 1
    country_name, 
    SUM(debt) AS total_debt
FROM international_debt_clean
GROUP BY country_name
ORDER BY total_debt DESC;

## 5. Monto promedio de deuda en todos los indicadores
<p>Vemos entonces que es <em>China</em>. Podemos encontrar un desglose más detallado de las deudas de China <a href="https://datatopics.worldbank.org/debt/ids/country/CHN">aquí</a>. </p>
<p>Ahora tenemos una breve descripción general del conjunto de datos y algunas de sus estadísticas resumidas. Además, tenemos una idea de los diferentes indicadores de deuda en los que los países deben sus deudas. Podemos investigar aún más para averiguar en promedio cuánta deuda debe un país. Esto nos dará una mejor idea de la distribución de la cantidad de deuda en diferentes indicadores.</p>

```sql
SELECT TOP 10
    indicator_code AS debt_indicator,
    indicator_name,
    AVG(debt) AS average_debt
FROM international_debt_clean
GROUP BY indicator_code, indicator_name
ORDER BY average_debt DESC;

## 6. El mayor monto de reembolsos de capital
<p>Podemos ver que el indicador <code>DT.AMT.DLXF.CD</code> encabeza el gráfico de deuda promedio. Esta categoría incluye el reembolso de deudas a largo plazo. Los países se endeudan a largo plazo para adquirir capital inmediato. Podemos encontrar más información sobre esta categoría <a href="https://datacatalog.worldbank.org/principal-repayments-external-debt-long-term-amt-current-us-0">aquí</a>. </p>
<p>Una observación interesante en el hallazgo anterior es que hay una gran diferencia en los montos de los indicadores después del segundo. Esto indica que los primeros dos indicadores podrían ser las categorías más severas en las que los países deben sus deudas.</p>
<p>Podemos investigar esto un poco más para averiguar qué país debe la mayor cantidad de deuda en la categoría de deudas a largo plazo (<code>DT.AMT.DLXF.CD</code>). Dado que no todos los países sufren el mismo tipo de perturbaciones económicas, este hallazgo nos permite comprender un poco más específicamente la condición económica de ese país en particular.

```sql
SELECT 
    country_name, 
    indicator_name
FROM international_debt_clean
WHERE debt = (SELECT 
                 MAX(debt)
             FROM international_debt_clean
             WHERE indicator_code = 'DT.AMT.DLXF.CD');

## 7. El indicador de deuda más común
<p>China tiene la mayor cantidad de deuda en la categoría de deuda a largo plazo (DT.AMT.DLXF.CD). Esto se verifica en el <a href="https://data.worldbank.org/indicator/DT.AMT.DLXF.CD?end=2018&most_recent_value_desc=true">Banco Mundial</a>. 
<p>Vimos que la deuda a largo plazo es la categoría más alta cuando se trata del monto promedio de deuda. Pero, ¿es el indicador más común en el que los países deben su deuda? </p>

```sql
SELECT TOP 20
    indicator_code, 
    COUNT(indicator_code) AS indicator_count
FROM international_debt_clean
GROUP BY indicator_code
ORDER BY indicator_count DESC, indicator_code DESC;

## 8. Otros problemas de deuda viables y conclusión
<p>Hay un total de seis indicadores de deuda en los que todos los países enumerados en nuestro conjunto de datos han contraído deuda. El indicador <code>DT.AMT.DLXF.CD</code> también está en la lista. Por lo tanto, esto nos da una pista de que todos estos países están sufriendo un problema económico común. Pero ese no es el final de la historia, sino una parte de ella. </p>
<p>Otro tema importante es la cantidad de deuda. Averigüamos la cantidad máxima de deuda en los indicadores junto con los respectivos nombres de los países. Con esto, estamos en condiciones de identificar otros problemas económicos plausibles por los que podría estar atravesando un país. De esta manera, podemos ver los indicadores de deuda en los que un país tiene la deuda más alta. </p>
<p>En este notebook, revisé la deuda de los países de todo el mundo. Extraje algunas estadísticas resumidas de los datos y descubrí algunos datos y cifras interesantes. También validé los hallazgos para asegurar de que las investigaciones sean correctas.

```sql
SELECT TOP 10
    country_name,
    MAX(debt) AS maximum_debt
FROM international_debt_clean
GROUP BY country_name
ORDER BY maximum_debt DESC;
